In [1]:
import os
from os.path import join

import numpy as np
import pandas as pd

In [2]:
DATA_PATH = join("..", "data", "Dyslexia_1_ready_data_fixations.csv")

In [3]:
def read_dataset(path: str = DATA_PATH):
    df = pd.read_csv(path, index_col="Unnamed: 0")
    
    df["SentenceID"] = df["Sentence_ID"]
    df["x"] = df["FIX_X"].apply(lambda x: float(x.replace(",", ".")))
    df["y"] = df["FIX_Y"].apply(lambda x: float(x.replace(",", ".")))
    df.x /= df.x.max()
    df.y /= df.y.max()
    df["duration"] = df["FIX_DURATION"].apply(lambda x: float(x) / 1000)
    df["timestamp"] = df.duration.cumsum()
    df["AOI_1"] = np.random.choice([0, 1], size=len(df))
    df["AOI_2"] = np.random.choice([11, 22, 33], size=len(df))
    df["AOI_3"] = np.random.choice(['A', 'B', 'C'], size=len(df))
    
    df.drop(["Word_Number", "IQ", "FIX_X", "FIX_Y", "FIX_DURATION", "Age", "Sentence_ID"], axis=1, inplace=True)
    return df

In [4]:
df = read_dataset()
df.head()

,SubjectID,Group,Sex,SentenceID,x,y,duration,timestamp,AOI_1,AOI_2,AOI_3
0,nnr8,3,fem,27,0.101158,0.726088,0.345,0.345,0,11,B
1,nnr8,3,fem,27,0.102406,0.754083,0.221,0.566,1,33,C
2,nnr8,3,fem,27,0.126049,0.755455,0.204,0.770,0,22,A
3,nnr8,3,fem,27,0.142134,0.761356,0.289,1.059,0,33,A
4,nnr8,3,fem,27,0.109963,0.766433,0.600,1.659,1,33,B


### 0. `Extractor` над нормальным датасетом.

In [5]:
import eyefeatures.features.stats as eye_stats
from eyefeatures.features.extractor import Extractor

sac_feats_stats = {
    'length': ['min', 'max'],
    'acceleration': ['mean']
}

sac_feats_stats_shift = {
    'acceleration': ['mean']
}

sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26_4,0.019488,0.092498,0.509739,-0.418699
nnr26_5,0.013345,0.080962,0.459010,-0.030698
nnr26_6,0.018656,0.112882,0.578460,-0.162751
nnr26_7,0.015868,0.118729,0.386805,-0.313379
nnr26_8,0.023546,0.090291,0.434029,-0.077838


### 1. Warnings & Errors.

В `BaseTransformer`, `Extractor`, и `StatsTransformer` добавлен параметр `warn: bool = True`, который контролирует показ всех `warning`-ов.

#### 1.1. Errors.

##### 1.1.1. Обработка `NaN`. Ожидается: warning в случае `NaN` в данных, `ValueError` в случае `NaN` в `pk`/`shift_pk`.

In [6]:
dfn = df.copy()
dfn.loc[0, "x"] = None
dfn.loc[0, "y"] = np.nan
extractor.fit_transform(dfn).head()

/tmp/ipykernel_27574/889556869.py:4: UserWarning: Group nnr8_27 has missing values. Dropping them.
  extractor.fit_transform(dfn).head()
100%|██████████| 1/1 [00:00<00:00,  2.10it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26_4,0.019488,0.092498,0.509739,-0.418699
nnr26_5,0.013345,0.080962,0.459010,-0.030698
nnr26_6,0.018656,0.112882,0.578460,-0.162751
nnr26_7,0.015868,0.118729,0.386805,-0.313379
nnr26_8,0.023546,0.090291,0.434029,-0.077838


In [7]:
dfn.loc[0, "SubjectID"] = None
try:
    extractor.fit_transform(dfn).head()
except ValueError as e:
    print(e)
    print("OK")

Found missing values in pk.
OK


#### 1.2. Warnings.

##### 1.2.1. Отсутствие warning-а при `warn=False`.

In [8]:
extractor_no_warn = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True,
    warn=False
)

dfn = df.copy()
dfn.loc[0, "x"] = None
dfn.loc[0, "y"] = np.nan

In [9]:
extractor.fit_transform(dfn).head();

/tmp/ipykernel_27574/1683100184.py:1: UserWarning: Group nnr8_27 has missing values. Dropping them.
  extractor.fit_transform(dfn).head();
100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


In [10]:
extractor_no_warn.fit_transform(dfn).head();

100%|██████████| 1/1 [00:00<00:00,  1.90it/s]


##### 1.2.2. Неизвестные значения в `shift_pk`. Ожидается: warning.

In [11]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True
)

In [12]:
dfn = df.copy()
dfn.loc[0, "SentenceID"] = -1

extractor.fit(df)
extractor.transform(dfn)

  0%|          | 0/1 [00:00<?, ?it/s]/home/vagiz/.cache/pypoetry/virtualenvs/eyefeatures-dVI54DNx-py3.12/lib/python3.12/site-packages/sklearn/utils/_set_output.py:313: UserWarning: Group -1 for shift_pk SentenceID was not seen during `fit`.Average across all values of SentenceID is used instead.
  data_to_wrap = f(self, X, *args, **kwargs)
100%|██████████| 1/1 [00:00<00:00,  2.14it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26_4,0.019488,0.092498,0.509739,-0.418699
nnr26_5,0.013345,0.080962,0.459010,-0.030698
nnr26_6,0.018656,0.112882,0.578460,-0.162751
nnr26_7,0.015868,0.118729,0.386805,-0.313379
nnr26_8,0.023546,0.090291,0.434029,-0.077838
...,...,...,...,...
nnr85_29,0.014600,0.140295,0.595447,-0.081194
nnr85_30,0.014928,0.417827,1.273437,0.694042
nnr85_31,0.034824,0.161574,0.607337,0.012661
nnr85_32,0.013381,0.140145,0.725873,0.276734


### 2. Несколько `shift_pk` и проверка, что можно не подмножество `pk`.

#### 2.0. Дефолтные случаи.

##### 2.0.0. Дефолтный случай `pk=None`, `shift_features=None`. Ожидается: одна строка и 3 фичи.

In [13]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    # shift_features=sac_feats_stats_shift,
    # shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    # pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 678.58it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean
0,0.000882,0.891518,10.213596


##### 2.0.1. Дефолтный случай `pk=...`, `shift_features=None`. Ожидается: 3 фичи.

In [14]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    # shift_features=sac_feats_stats_shift,
    # shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID', 'SentenceID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  2.85it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean
nnr26_4,0.019488,0.092498,0.509739
nnr26_5,0.013345,0.080962,0.459010
nnr26_6,0.018656,0.112882,0.578460
nnr26_7,0.015868,0.118729,0.386805
nnr26_8,0.023546,0.090291,0.434029


#### 2.1. Можно `shift_pk` не подмножество `shift_features`.

##### 2.1.1. Можно `shift_pk` не подмножество `shift_features`. Ожидается: 3 фичи + 1 shift фича.

In [15]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=['SentenceID']
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 24.87it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID
nnr26,0.006898,0.879102,0.739731,-1.657837
nnr27,0.007935,0.868788,0.504197,-1.125991
nnr33,0.002138,0.866240,6.324986,4.960990
nnr36,0.001416,0.784169,0.487095,-1.739695
nnr41,0.002338,0.857037,150.445112,147.904599


#### 2.2. Комбинации `shift_pk` и `shift_features`.

##### 2.2.1. Можно несколько `shift_pk` на один `shift_features`. Ожидается: 3 фичи + 3 shift фичи.

In [16]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID', 'SubjectID'], ['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 13.58it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID_SubjectID,sac_acceleration_mean_shift_SentenceID,sac_acceleration_mean_shift_SubjectID
nnr26,0.006898,0.879102,0.739731,0.172832,-1.657837,2.220446e-16
nnr27,0.007935,0.868788,0.504197,0.158958,-1.125991,0.000000e+00
nnr33,0.002138,0.866240,6.324986,5.701891,4.960990,0.000000e+00
nnr36,0.001416,0.784169,0.487095,0.098271,-1.739695,0.000000e+00
nnr41,0.002338,0.857037,150.445112,149.332152,147.904599,0.000000e+00


##### 2.2.2. Можно несколько `shift_pk` и несколько `shift_features`. Ожидается: 3 фичи + 3 shift фичи.

In [17]:
sf = eye_stats.SaccadeFeatures(
    features_stats={'length': ['min', 'max', 'median']},
    shift_features=({'length': ['min']}, {'length': ['max', 'median']}),
    shift_pk=(['SentenceID', 'SubjectID'], ['SentenceID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 15.78it/s]


,sac_length_min,sac_length_max,sac_length_median,sac_length_min_shift_SentenceID_SubjectID,sac_length_max_shift_SentenceID,sac_length_median_shift_SentenceID
nnr26,0.006898,0.879102,0.043258,-0.008151,0.181826,0.005024
nnr27,0.007935,0.868788,0.042489,-0.005300,0.166603,0.004397
nnr33,0.002138,0.866240,0.037203,-0.010555,0.163130,-0.000845
nnr36,0.001416,0.784169,0.025004,-0.007431,0.086215,-0.012591
nnr41,0.002338,0.857037,0.049938,-0.011231,0.162888,0.011563


### 3. Можно несколько разных `AOI` колонок.

#### 3.0. Дефолтные случаи.

##### 3.0.1. `aoi=None`. Ожидается: 3 фичи + 2 shift фичи.

In [18]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi=None,
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00, 17.22it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID,sac_acceleration_mean_shift_SubjectID
nnr26,0.006898,0.879102,0.739731,-1.657837,2.220446e-16
nnr27,0.007935,0.868788,0.504197,-1.125991,0.000000e+00
nnr33,0.002138,0.866240,6.324986,4.960990,0.000000e+00
nnr36,0.001416,0.784169,0.487095,-1.739695,0.000000e+00
nnr41,0.002338,0.857037,150.445112,147.904599,0.000000e+00


#### 3.1. Одно и более значений `AOI` + `calc_without_aoi`.

##### 3.1.1. Одно значение `AOI`. Ожидается: (3 фичи + 2 shift фичи) x 2 для `AOI_1=0` и `AOI_1=1`.

In [19]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi='AOI_1',
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  7.58it/s]


,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_max,sac_acceleration_AOI_1[0]_mean,sac_acceleration_AOI_1[0]_mean_shift_SentenceID,sac_acceleration_AOI_1[0]_mean_shift_SubjectID,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_max,sac_acceleration_AOI_1[1]_mean,sac_acceleration_AOI_1[1]_mean_shift_SentenceID,sac_acceleration_AOI_1[1]_mean_shift_SubjectID
nnr26,0.010187,0.879102,0.702945,-0.009890,0.0,0.006898,0.707649,0.656637,0.006708,0.0
nnr27,0.009836,0.716514,0.452859,-0.242391,0.0,0.014549,0.704578,0.570162,-0.062861,0.0
nnr33,0.009231,0.723296,1.433318,0.671181,0.0,0.002138,0.723941,0.856447,0.216838,0.0
nnr36,0.004829,0.784169,0.600477,-0.027443,0.0,0.004899,0.704900,0.490630,-0.133978,0.0
nnr41,0.005504,0.702948,2.209051,1.471555,0.0,0.004879,0.857037,19.822000,19.167571,0.0


##### 3.1.2. Одно значение `AOI`. Ожидается: (3 фичи + 2 shift фичи) x (1 + 2) для `calc_with_aoi=True`, `AOI_1=0` и `AOI_1=1`.

In [20]:
sf = eye_stats.SaccadeFeatures(
    features_stats=sac_feats_stats,
    shift_features=sac_feats_stats_shift,
    shift_pk=(['SentenceID'], ['SubjectID']),
    calc_without_aoi=True
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi='AOI_1',
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  5.17it/s]


,sac_length_min,sac_length_max,sac_acceleration_mean,sac_acceleration_mean_shift_SentenceID,sac_acceleration_mean_shift_SubjectID,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_max,sac_acceleration_AOI_1[0]_mean,sac_acceleration_AOI_1[0]_mean_shift_SentenceID,sac_acceleration_AOI_1[0]_mean_shift_SubjectID,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_max,sac_acceleration_AOI_1[1]_mean,sac_acceleration_AOI_1[1]_mean_shift_SentenceID,sac_acceleration_AOI_1[1]_mean_shift_SubjectID
nnr26,0.006898,0.879102,0.739731,-1.657837,2.220446e-16,0.010187,0.879102,0.702945,-0.009890,0.0,0.006898,0.707649,0.656637,0.006708,0.0
nnr27,0.007935,0.868788,0.504197,-1.125991,0.000000e+00,0.009836,0.716514,0.452859,-0.242391,0.0,0.014549,0.704578,0.570162,-0.062861,0.0
nnr33,0.002138,0.866240,6.324986,4.960990,0.000000e+00,0.009231,0.723296,1.433318,0.671181,0.0,0.002138,0.723941,0.856447,0.216838,0.0
nnr36,0.001416,0.784169,0.487095,-1.739695,0.000000e+00,0.004829,0.784169,0.600477,-0.027443,0.0,0.004899,0.704900,0.490630,-0.133978,0.0
nnr41,0.002338,0.857037,150.445112,147.904599,0.000000e+00,0.005504,0.702948,2.209051,1.471555,0.0,0.004879,0.857037,19.822000,19.167571,0.0


##### 3.1.3. Три значения `AOI`. Ожидается: (1 фича + 2 shift фичи) x (2 + 3 + 3) = 24 для `AOI_1` $\in [0, 1]$, `AOI_2` $\in [11, 22, 33]$, `AOI_3` $\in [A, B, C]$.


In [21]:
sf = eye_stats.SaccadeFeatures(
    features_stats={'length': ['min']},
    shift_features={'length': ['min']},
    shift_pk=(['SentenceID'], ['SubjectID'])
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi=['AOI_1', 'AOI_2', 'AOI_3'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_min_shift_SentenceID,sac_length_AOI_1[0]_min_shift_SubjectID,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_min_shift_SentenceID,sac_length_AOI_1[1]_min_shift_SubjectID,sac_length_AOI_2[11]_min,sac_length_AOI_2[11]_min_shift_SentenceID,sac_length_AOI_2[11]_min_shift_SubjectID,sac_length_AOI_2[33]_min,...,sac_length_AOI_2[22]_min_shift_SubjectID,sac_length_AOI_3[B]_min,sac_length_AOI_3[B]_min_shift_SentenceID,sac_length_AOI_3[B]_min_shift_SubjectID,sac_length_AOI_3[C]_min,sac_length_AOI_3[C]_min_shift_SentenceID,sac_length_AOI_3[C]_min_shift_SubjectID,sac_length_AOI_3[A]_min,sac_length_AOI_3[A]_min_shift_SentenceID,sac_length_AOI_3[A]_min_shift_SubjectID
nnr26,0.010187,0.003042,0.0,0.006898,0.000846,0.0,0.009671,0.000579,0.0,0.010186,...,0.0,0.012470,0.003794,0.0,0.009671,0.000822,0.0,0.011133,0.001897,0.0
nnr27,0.009836,0.002738,0.0,0.014549,0.008204,0.0,0.010210,0.001029,0.0,0.011390,...,0.0,0.007935,-0.000857,0.0,0.010210,0.001217,0.0,0.012107,0.003335,0.0
nnr33,0.009231,0.002047,0.0,0.002138,-0.004199,0.0,0.002138,-0.007016,0.0,0.009477,...,0.0,0.002138,-0.006638,0.0,0.004336,-0.004441,0.0,0.010616,0.001543,0.0
nnr36,0.004829,-0.001832,0.0,0.004899,-0.000996,0.0,0.009618,0.000612,0.0,0.004243,...,0.0,0.006172,-0.002552,0.0,0.009221,0.000002,0.0,0.005381,-0.003927,0.0
nnr41,0.005504,-0.001374,0.0,0.004879,-0.001391,0.0,0.016214,0.007139,0.0,0.006884,...,0.0,0.005504,-0.003164,0.0,0.009950,0.000937,0.0,0.008209,-0.001186,0.0


##### 3.1.4. Три значения `AOI`. Ожидается: (1 фича + 2 shift фичи) x (2 + 3 + 3 + 1) = 27 для `AOI_1` $\in [0, 1]$, `AOI_2` $\in [11, 22, 33]$, `AOI_3` $\in [A, B, C]$, `calc_without_aoi=True`.


In [22]:
sf = eye_stats.SaccadeFeatures(
    features_stats={'length': ['min']},
    shift_features={'length': ['min']},
    shift_pk=(['SentenceID'], ['SubjectID']),
    calc_without_aoi=True
)

extractor = Extractor(
    features=[sf],
    x='x',
    y='y',
    t='timestamp',
    duration='duration',
    # dispersion='dispersion',
    # path_pk=['SubjectID'],
    aoi=['AOI_1', 'AOI_2', 'AOI_3'],
    pk=['SubjectID'],
    return_df=True
)

extractor.fit_transform(df).head()

100%|██████████| 1/1 [00:00<00:00,  1.82it/s]


,sac_length_min,sac_length_min_shift_SentenceID,sac_length_min_shift_SubjectID,sac_length_AOI_1[0]_min,sac_length_AOI_1[0]_min_shift_SentenceID,sac_length_AOI_1[0]_min_shift_SubjectID,sac_length_AOI_1[1]_min,sac_length_AOI_1[1]_min_shift_SentenceID,sac_length_AOI_1[1]_min_shift_SubjectID,sac_length_AOI_2[11]_min,...,sac_length_AOI_2[22]_min_shift_SubjectID,sac_length_AOI_3[B]_min,sac_length_AOI_3[B]_min_shift_SentenceID,sac_length_AOI_3[B]_min_shift_SubjectID,sac_length_AOI_3[C]_min,sac_length_AOI_3[C]_min_shift_SentenceID,sac_length_AOI_3[C]_min_shift_SubjectID,sac_length_AOI_3[A]_min,sac_length_AOI_3[A]_min_shift_SentenceID,sac_length_AOI_3[A]_min_shift_SubjectID
nnr26,0.006898,0.003611,0.0,0.010187,0.003042,0.0,0.006898,0.000846,0.0,0.009671,...,0.0,0.012470,0.003794,0.0,0.009671,0.000822,0.0,0.011133,0.001897,0.0
nnr27,0.007935,0.004490,0.0,0.009836,0.002738,0.0,0.014549,0.008204,0.0,0.010210,...,0.0,0.007935,-0.000857,0.0,0.010210,0.001217,0.0,0.012107,0.003335,0.0
nnr33,0.002138,-0.001151,0.0,0.009231,0.002047,0.0,0.002138,-0.004199,0.0,0.002138,...,0.0,0.002138,-0.006638,0.0,0.004336,-0.004441,0.0,0.010616,0.001543,0.0
nnr36,0.001416,-0.001555,0.0,0.004829,-0.001832,0.0,0.004899,-0.000996,0.0,0.009618,...,0.0,0.006172,-0.002552,0.0,0.009221,0.000002,0.0,0.005381,-0.003927,0.0
nnr41,0.002338,-0.000950,0.0,0.005504,-0.001374,0.0,0.004879,-0.001391,0.0,0.016214,...,0.0,0.005504,-0.003164,0.0,0.009950,0.000937,0.0,0.008209,-0.001186,0.0


#### Проверка названий фичей

In [23]:
extractor.features[0].feature_names_in_

['sac_length_min',
 'sac_length_min_shift_SentenceID',
 'sac_length_min_shift_SubjectID',
 'sac_length_AOI_1[0]_min',
 'sac_length_AOI_1[0]_min_shift_SentenceID',
 'sac_length_AOI_1[0]_min_shift_SubjectID',
 'sac_length_AOI_1[1]_min',
 'sac_length_AOI_1[1]_min_shift_SentenceID',
 'sac_length_AOI_1[1]_min_shift_SubjectID',
 'sac_length_AOI_2[11]_min',
 'sac_length_AOI_2[11]_min_shift_SentenceID',
 'sac_length_AOI_2[11]_min_shift_SubjectID',
 'sac_length_AOI_2[33]_min',
 'sac_length_AOI_2[33]_min_shift_SentenceID',
 'sac_length_AOI_2[33]_min_shift_SubjectID',
 'sac_length_AOI_2[22]_min',
 'sac_length_AOI_2[22]_min_shift_SentenceID',
 'sac_length_AOI_2[22]_min_shift_SubjectID',
 'sac_length_AOI_3[B]_min',
 'sac_length_AOI_3[B]_min_shift_SentenceID',
 'sac_length_AOI_3[B]_min_shift_SubjectID',
 'sac_length_AOI_3[C]_min',
 'sac_length_AOI_3[C]_min_shift_SentenceID',
 'sac_length_AOI_3[C]_min_shift_SubjectID',
 'sac_length_AOI_3[A]_min',
 'sac_length_AOI_3[A]_min_shift_SentenceID',
 'sac_len